In [ ]:
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt

## `SciPy.integrate`: Numerická integrace a řešení diferenciálních rovnic

### Vyčíslení určitého integrálu
Numerical evaluation of a function of the type
Často potřebujeme numericky vyčíslit určitý integrál, tj.

$\displaystyle \int_a^b f(x) {\rm d}x$

Numerické integraci se často říká kvadratura, anglicky *quadrature*. Podle toho se jmenují o funkce v modulu `scipy.integrate`, např. `quad`, `dblquad`, `tplquad` nebo obecné `nquad`.

In [ ]:
import scipy.integrate as spi


Zkusíme spočítat jednodychý integrál:

$\displaystyle \int_0^1 x {\rm d}x$

In [ ]:
val, abserr = spi.quad(lambda x: x, 0, 1)
print(f"výsledek = {val:g} ± {abserr:.2g}")

Můžeme dokonce pracovat s nekonečnými mezemi.

$$\displaystyle \int_{-\infty}^\infty e^{-x^2} {\rm d}x$$

In [ ]:
val, abserr = spi.quad(lambda x: np.exp(-x ** 2), -np.inf, np.inf)
print(f"výsledek = {val:g} ± {abserr:.2g}")
print(f"rozdíl od přesné hodnoty (√π) = {val - np.sqrt(np.pi):g}")


## Obyčejné diferenciální rovnice (ODR)

`scipy.integrate` (ano, řešení ODR je v tomto modulu, protože řešením ODR je určitý integrál) obsahuje `odeint`, které je jednodušší, a objektové rozhraní `ode`, které umožňuje větší kontrolu.


ODR (nebo jejich soustava) je často zadána jako

$y' = f(y, t)$

s počátečními podmínkami

$y(t=0) = y_0$



### Ukázka: jednoduché kyvadlo

Rovnice jednoduchého fyzikálního kyvadla je

Zrychlení kyvadla $\theta ''$ závisí na pozici $\theta$ a gravitační konstanty $g$ a délky kyvadla $L$.

$\displaystyle {\theta ''} = - \frac{g}{L}\theta$

Řešení je známé, použijeme jej pro kontrolu:

$\displaystyle {\theta} = \theta_0 \cos\left( \sqrt{\frac{g}{L}} t \right) $


#### Numerické řešení

Pro řešení pomocí `odeint` musíme vyrobit funkci `f`:
- přijímá dvě proměnné: aktuální stav `y` a čas `t`
    - `y` je pole s funkční hodnotou a všemi derivacemi kromě nejvyšší: [$\theta$, $\theta '$]
    - `t` je skalár, aktuální čas
- vrací pole všech derivací: [$\theta '$, $\theta ''$]


`odeint` pak lze použít jednoduše:

    y_t = odeint(f, y_0, t)

kde `t` je předem zadané pole časových bodů, ve kterých požadujeme řešení a `y_0` je pole počátrčních podmínek.


In [ ]:
from scipy.constants import g

L = 0.5
m = 0.1


def f_pendulum(x, t):
    """
    Pravá strana diferenciální rovnice kyvadla.
    Tato funkce vrací derivace úhlu a úhlové rychlosti kyvadla vzhledem k času.

    Parametry:
    x: List obsahující funkční hodnotu úhlu a úhlové rychlosti kyvadla (funkce, derivace)
    t: čas (v tomto případě není přímo třeba, např. kdyby některý parametr závisel na čase)

    Vrací:
    List hodnot: [d_theta_dt, d_dtheta_dt] 
        - dtheta: První derivace úhlu theta, což je úhlová rychlost kyvadla
        - ddtheta: Druhá derivace úhlu theta, což je úhlové zrychlení kyvadla
    """
    theta, dtheta = x[0], x[1]  # Rozbalení úhlu a úhlové rychlosti

    ddtheta = - g / L * theta  # Druhá derivace úhlu theta, což odpovídá Newtonovu zákonu pro malé výchylky

    return [dtheta, ddtheta]

In [ ]:
# počáteční stav = pozice a rychlost
x0 = [np.pi / 8, 0]
# časy pro řešení
t = np.linspace(0, 10, 250)
# a konečně řešení
x = spi.odeint(f_pendulum, x0, t)

In [ ]:
# analytické řešení
x_anal = x0[0] * np.cos(np.sqrt(g / L) * t)

In [ ]:
plt.plot(t, x[:, 0], 'r', label=r"$\theta$")
plt.plot(t, x_anal, 'k--', label=u"přesné řešení")
plt.legend()
plt.xlabel("Čas [s]")
plt.ylabel("Poloha")